In [1]:
from easydict import EasyDict as edict
from PIL import Image
import torch, torchvision
from torchvision.datasets.video_utils import VideoClips
import os, json
import numpy as np
from torch.utils.data import Dataset, DataLoader
from time import time
import pickle as pkl
from collections import Counter as C

_C = edict()
_C.NETWORK = edict()
_C.NETWORK.BACKBONE = "res18"
_C.NETWORK.BACKBONE_LOAD_PRETRAINED = True

_C.NETWORK.TEMPORAL_MLP_DIMS = 512
_C.NETWORK.TEMPORAL_MLP_ACTIVATION = "LeakyReLU"

_C.NETWORK.TRANSFORMER_DIMS = 512
_C.NETWORK.TRANSFORMER_HEADS = 8
_C.NETWORK.TRANSFORMER_ENCODER_CNT = 8
_C.NETWORK.TRANSFORMER_DROPOUT = 0.1
_C.NETWORK.TRANSFORMER_FEEDFORWARD_DIMS = 2048

_C.NETWORK.POSITIONAL_DROPOUT = 0.1
_C.NETWORK.NUM_CLASSES = 700
_C

{'NETWORK': {'BACKBONE': 'res18',
  'BACKBONE_LOAD_PRETRAINED': True,
  'TEMPORAL_MLP_DIMS': 512,
  'TEMPORAL_MLP_ACTIVATION': 'LeakyReLU',
  'TRANSFORMER_DIMS': 512,
  'TRANSFORMER_HEADS': 8,
  'TRANSFORMER_ENCODER_CNT': 8,
  'TRANSFORMER_DROPOUT': 0.1,
  'TRANSFORMER_FEEDFORWARD_DIMS': 2048,
  'POSITIONAL_DROPOUT': 0.1,
  'NUM_CLASSES': 700}}

In [2]:
basepath = '/data/datasets/kinetics700_2020/'
train_dataset = json.load(open(basepath + "labels/train.json"))

In [3]:
list(train_dataset.items())[990]

('-7N2pxxB2f4',
 {'annotations': {'label': 'frying vegetables', 'segment': [267.0, 277.0]},
  'duration': 10.0,
  'subset': 'train',
  'url': 'https://www.youtube.com/watch?v=-7N2pxxB2f4',
  'nb_frames': '250',
  'hw': [1080, 1920],
  'true_duration': '10.000000'})

In [4]:
from time import time
def bench(cmd,iter):
    tic = time()
    for i in range(iter):
        cmd()
    print(time() - tic)

In [5]:
class load_and_dump(Dataset):
    def __init__(self, dataset, video_folder, save_folder):
        super(load_and_dump, self).__init__()
        assert video_folder[-1] == '/'
        assert save_folder[-1] == '/'

        self.dataset = list(dataset.items())
        self.video_path = video_folder
        self.save_path = save_folder
        assert self.video_path != self.save_path

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        k, v = self.dataset[idx]

        vid_path = self.video_path + k + '.mp4'
        save_path = self.save_path + k + '.mp4' 

        # Remove Audio, Subtitles ; Resize Video to 224X224 and 10 fps
        os.system("ffmpeg -i " + vid_path + ' -vf "fps=10,scale=224:224" -an -sn ' + save_path)
        
        return k

In [ ]:
collate = lambda x: x
batch_size = 60
dataset = DataLoader(load_and_dump(dict(list(train_dataset.items())),
                                    basepath + 'train/',
                                    basepath + 'train_cache/',
                                ),
                     batch_size=batch_size,
                     num_workers=batch_size,
                     collate_fn=collate
                    )
probes = []
for i, batch in enumerate(dataset):
    if i % 10 == 0:
        print(i*50,'/', len(train_dataset))
    probes.append(batch)

0 / 523099
500 / 523099
1000 / 523099
1500 / 523099
2000 / 523099
2500 / 523099
3000 / 523099
3500 / 523099
4000 / 523099
4500 / 523099
5000 / 523099
5500 / 523099
6000 / 523099
6500 / 523099
7000 / 523099
7500 / 523099
8000 / 523099
8500 / 523099
9000 / 523099
9500 / 523099
10000 / 523099
10500 / 523099
11000 / 523099
11500 / 523099
12000 / 523099
12500 / 523099
13000 / 523099
13500 / 523099
14000 / 523099
14500 / 523099
15000 / 523099
15500 / 523099
16000 / 523099
16500 / 523099
17000 / 523099
17500 / 523099
18000 / 523099
18500 / 523099
19000 / 523099
19500 / 523099
20000 / 523099
20500 / 523099
21000 / 523099
21500 / 523099
22000 / 523099
22500 / 523099
23000 / 523099
23500 / 523099
24000 / 523099
24500 / 523099
25000 / 523099
25500 / 523099
26000 / 523099
26500 / 523099
27000 / 523099
27500 / 523099
28000 / 523099
28500 / 523099
29000 / 523099
29500 / 523099
30000 / 523099
30500 / 523099
31000 / 523099
31500 / 523099
32000 / 523099
32500 / 523099
33000 / 523099
33500 / 523099
3400

In [ ]:
len(probes)

In [ ]:
probes[0]